In [71]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

"""
MAC Instructions for ChromeDriver
HomeBrew Download ChromeDriver
Run xattr -d com.apple.quarantine ChromeDriver in your folder with chromedriver.exe -- itll be in homebrew folder
Should Work
"""

"""
MAC Instructions for LangChain
run !pip install openapi-pydantic 
"""


url = "https://sportsdata.io/developers/api-documentation/nfl"  # your URL here 

# Create a new Chrome browser instance
driver = webdriver.Chrome()

# Navigate to the application home page
driver.get(url)

time.sleep(5)



html_page = driver.page_source

driver.close()




In [135]:
soup = BeautifulSoup(html_page, 'html.parser')
div_tags = soup.select('div.operation-block')
data = []
pattern = r"Tags:\n"

for div in div_tags:
    operation_title = div.select_one('div.operation-title')
    operations = div.select('div.ng-scope')
    description = operations[0].text.strip()
    tags = operations[1].text.strip()
    tags = re.sub(pattern, '', tags).strip()
    endpoint = div.select('a')[0].text
    
    
    param_table = div.select_one('div.param-table')
    if(param_table):
   
        param_rows = param_table.select('div.param-row')
        param_rows = param_rows[2:]
        params =[]

       
        for row in param_rows:
            param_name = row.select_one('div.param-name').text.strip()
            param_type = row.select('div.ng-binding')[1].text.strip()
            param_required = row.select('div.ng-binding')[2].text.strip()
            description_ = row.select('div.ng-binding')[3].text.strip()
            params.append({"name": param_name, "type":param_type,"required":param_required, "description":description_})

            
        





        if operation_title: 
            data.append({"title":operation_title.text.strip(), "description":description, "tags":tags,"endpoint":endpoint, "params":params }) 


In [136]:
data

[{'title': 'Push Authentication (Game State)',
  'description': 'Returns the push token for the game state channel',
  'tags': 'Game State Feeds',
  'endpoint': 'https://api.sportsdata.io/v3/nfl/scores/json/pushauthgamestate',
  'params': []},
 {'title': 'Are Games In Progress',
  'description': 'Returns true if there is at least one game being played at the time of the request or false if there are none.',
  'tags': 'Schedules & Game Day Info Feeds',
  'endpoint': 'https://api.sportsdata.io/v3/nfl/scores/json/AreAnyGamesInProgress',
  'params': []},
 {'title': 'Bye Weeks',
  'description': 'Get bye weeks for the teams during a specified NFL season.',
  'tags': 'Schedules & Game Day Info Feeds',
  'endpoint': 'https://api.sportsdata.io/v3/nfl/scores/json/Byes/{season}',
  'params': [{'name': 'season',
    'type': 'string',
    'required': 'true',
    'description': ''}]},
 {'title': 'Season Current',
  'description': 'Year of the current NFL season. This value changes at the start of t